In [20]:
from dotenv import load_dotenv
from os import getenv
import requests
from pinecone import Pinecone, ServerlessSpec

# Load environment variables from a .env file
load_dotenv()





True

In [15]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws",region="us-east-1")
)

In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

In [21]:
processed_data=[]

from openai import OpenAI

# Initialize OpenAI client using OpenRouter's base URL and API key from environment variable
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("OPENROUTER_API_KEY"),
)

# Create a chat completion request using the LLaMA 3.1 model
completion = client.chat.completions.create(
  model="meta-llama/llama-3.1-8b-instruct:free",
  messages=[
    {
      "role": "user",
      "content": "Say this is a test",
    },
  ],
)

# Print the response content
print(completion.choices[0].message['content'])


# for review in data['reviews']:
    


ImportError: dlopen(/Users/duaatariq/opt/anaconda3/envs/rag/lib/python3.10/site-packages/pydantic_core/_pydantic_core.cpython-310-darwin.so, 0x0002): tried: '/Users/duaatariq/opt/anaconda3/envs/rag/lib/python3.10/site-packages/pydantic_core/_pydantic_core.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/duaatariq/opt/anaconda3/envs/rag/lib/python3.10/site-packages/pydantic_core/_pydantic_core.cpython-310-darwin.so' (no such file), '/Users/duaatariq/opt/anaconda3/envs/rag/lib/python3.10/site-packages/pydantic_core/_pydantic_core.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))